# Strategies in Federated Learning

Welcome to the next part of the federated learning tutorial. In previous parts of this tutorial, we introduced federated learning with PyTorch and Flower ([part 1](https://flower.dev/docs/tutorial/Flower-1-Intro-to-FL-PyTorch.html)).

In this notebook, we'll begin to customize the federated learning system we built in the introductory notebook (again, using [Flower](https://flower.dev/) and [PyTorch](https://pytorch.org/)).

> [Star Flower on GitHub](https://github.com/adap/flower) ⭐️ and join the Flower community on Slack to connect, ask questions, and get help: [Join Slack](https://flower.dev/join-slack) 🌼 We'd love to hear from you in the `#introductions` channel! And if anything is unclear, head over to the `#questions` channel.

Let's move beyond FedAvg with Flower Strategies!

## Preparation

Before we begin with the actual code, let's make sure that we have everything we need.

### Installing dependencies

First, we install the necessary packages:

Now that we have all dependencies installed, we can import everything we need for this tutorial:

In [1]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

import flwr as fl

DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

Training on cpu using PyTorch 2.0.0 and Flower 1.4.0


It is possible to switch to a runtime that has GPU acceleration enabled (on Google Colab: `Runtime > Change runtime type > Hardware acclerator: GPU > Save`). Note, however, that Google Colab is not always able to offer GPU acceleration. If you see an error related to GPU availability in one of the following sections, consider switching back to CPU-based execution by setting `DEVICE = torch.device("cpu")`. If the runtime has GPU acceleration enabled, you should see the output `Training on cuda`, otherwise it'll say `Training on cpu`.

### Data loading

Let's now load the CIFAR-10 training and test set, partition them into ten smaller datasets (each split into training and validation set), and wrap everything in their own `DataLoader`. We introduce a new parameter `num_clients` which allows us to call `load_datasets` with different numbers of clients.

In [2]:
NUM_CLIENTS = 10


def load_datasets(num_clients: int):
    # Download and transform CIFAR-10 (train and test)
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    trainset = CIFAR10("./dataset", train=True, download=True, transform=transform)
    testset = CIFAR10("./dataset", train=False, download=True, transform=transform)

    # Split training set into `num_clients` partitions to simulate different local datasets
    partition_size = len(trainset) // num_clients
    lengths = [partition_size] * num_clients
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=32, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=32))
    testloader = DataLoader(testset, batch_size=32)
    return trainloaders, valloaders, testloader


trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

Files already downloaded and verified
Files already downloaded and verified


### Model training/evaluation

Let's continue with the usual model definition (including `set_parameters` and `get_parameters`), training and test functions:

In [3]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def train(net, trainloader, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

### Flower client

To implement the Flower client, we (again) create a subclass of `flwr.client.NumPyClient` and implement the three methods `get_parameters`, `fit`, and `evaluate`. Here, we also pass the `cid` to the client and use it log additional details:

In [4]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

## Strategy customization

So far, everything should look familiar if you've worked through the introductory notebook. With that, we're ready to introduce a number of new features. 

### Server-side parameter **initialization**

Flower, by default, initializes the global model by asking one random client for the initial parameters. In many cases, we want more control over parameter initialization though. Flower therefore allows you to directly pass the initial parameters to the Strategy:

In [5]:
# Create an instance of the model and get the parameters
params = get_parameters(Net())

# Pass parameters to the Strategy for server-side parameter initialization
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(params),
)

# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-04-22 07:02:11,234 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-04-22 07:02:12,835	INFO worker.py:1553 -- Started a local Ray instance.
INFO flwr 2023-04-22 07:02:13,347 | app.py:180 | Flower VCE: Ray initialized with resources: {'CPU': 16.0, 'GPU': 1.0, 'memory': 81449971917.0, 'accelerator_type:RTX': 1.0, 'object_store_memory': 39192845107.0, 'node:172.17.0.2': 1.0}
INFO flwr 2023-04-22 07:02:13,348 | server.py:86 | Initializing global parameters
INFO flwr 2023-04-22 07:02:13,348 | server.py:269 | Using initial parameters provided by strategy
INFO flwr 2023-04-22 07:02:13,348 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-04-22 07:02:13,349 | server.py:101 | FL starting
DEBUG flwr 2023-04-22 07:02:13,349 | server.py:218 | fit_round 1: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=26190) [Client 7] fit, config: {}
(launch_and_fit pid=26178) [Client 1] fit, config: {}
(launch_and_fit pid=26181) [Client 2] fit, config: {}


DEBUG flwr 2023-04-22 07:02:16,933 | server.py:232 | fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-04-22 07:02:16,937 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-04-22 07:02:16,937 | server.py:168 | evaluate_round 1: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=26178) Epoch 1: train loss 0.06525014340877533, accuracy 0.22977777777777778


DEBUG flwr 2023-04-22 07:02:18,490 | server.py:182 | evaluate_round 1 received 3 results and 0 failures
WARNING flwr 2023-04-22 07:02:18,491 | fedavg.py:274 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2023-04-22 07:02:18,491 | server.py:218 | fit_round 2: strategy sampled 3 clients (out of 10)


...

DEBUG flwr 2023-04-22 07:02:22,794 | server.py:182 | evaluate_round 2 received 3 results and 0 failures
DEBUG flwr 2023-04-22 07:02:22,795 | server.py:218 | fit_round 3: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=26190) Epoch 1: train loss 0.058349113911390305, accuracy 0.3051111111111111
(launch_and_fit pid=26181) Epoch 1: train loss 0.05785178765654564, accuracy 0.3091111111111111
(launch_and_evaluate pid=26178) [Client 5] evaluate, config: {}
(launch_and_evaluate pid=26190) [Client 9] evaluate, config: {}
(launch_and_evaluate pid=26181) [Client 3] evaluate, config: {}
(launch_and_fit pid=26178) [Client 1] fit, config: {}
(launch_and_fit pid=26190) [Client 0] fit, config: {}
(launch_and_fit pid=26181) [Client 8] fit, config: {}


DEBUG flwr 2023-04-22 07:02:25,554 | server.py:232 | fit_round 3 received 3 results and 0 failures
DEBUG flwr 2023-04-22 07:02:25,558 | server.py:168 | evaluate_round 3: strategy sampled 3 clients (out of 10)
DEBUG flwr 2023-04-22 07:02:27,118 | server.py:182 | evaluate_round 3 received 3 results and 0 failures
INFO flwr 2023-04-22 07:02:27,118 | server.py:147 | FL finished in 13.769745540019358
INFO flwr 2023-04-22 07:02:27,119 | app.py:218 | app_fit: losses_distributed [(1, 0.06241602524121601), (2, 0.055532717386881515), (3, 0.05071285470326742)]
INFO flwr 2023-04-22 07:02:27,120 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-04-22 07:02:27,120 | app.py:220 | app_fit: metrics_distributed {}
INFO flwr 2023-04-22 07:02:27,120 | app.py:221 | app_fit: losses_centralized []
INFO flwr 2023-04-22 07:02:27,120 | app.py:222 | app_fit: metrics_centralized {}


(launch_and_fit pid=26178) Epoch 1: train loss 0.05285913124680519, accuracy 0.3731111111111111
(launch_and_fit pid=26190) Epoch 1: train loss 0.05304573103785515, accuracy 0.37377777777777776
(launch_and_fit pid=26181) Epoch 1: train loss 0.052951399236917496, accuracy 0.3771111111111111
(launch_and_evaluate pid=26190) [Client 6] evaluate, config: {}


History (loss, distributed):
	round 1: 0.06241602524121601
	round 2: 0.055532717386881515
	round 3: 0.05071285470326742

Passing `initial_parameters` to the `FedAvg` strategy prevents Flower from asking one of the clients for the initial parameters. If we look closely, we can see that the logs do not show any calls to the `FlowerClient.get_parameters` method.

### Starting with a customized strategy

We've seen the function `start_simulation` before. It accepts a number of arguments, amongst them the `client_fn` used to create `FlowerClient` instances, the number of clients to simulate `num_clients`, the number of rounds `num_rounds`, and the strategy.

The strategy encapsulates the federated learning approach/algorithm, for example, `FedAvg` or `FedAdagrad`. Let's try to use a different strategy this time:

In [6]:
# Create FedAdam strategy
strategy = fl.server.strategy.FedAdagrad(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

(launch_and_evaluate pid=26178) [Client 3] evaluate, config: {}
(launch_and_evaluate pid=26181) [Client 7] evaluate, config: {}


INFO flwr 2023-04-22 07:02:27,328 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-04-22 07:02:31,114	INFO worker.py:1553 -- Started a local Ray instance.
INFO flwr 2023-04-22 07:02:31,627 | app.py:180 | Flower VCE: Ray initialized with resources: {'GPU': 1.0, 'object_store_memory': 39191371776.0, 'accelerator_type:RTX': 1.0, 'CPU': 16.0, 'memory': 81446534144.0, 'node:172.17.0.2': 1.0}
INFO flwr 2023-04-22 07:02:31,628 | server.py:86 | Initializing global parameters
INFO flwr 2023-04-22 07:02:31,628 | server.py:269 | Using initial parameters provided by strategy
INFO flwr 2023-04-22 07:02:31,628 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-04-22 07:02:31,629 | server.py:101 | FL starting
DEBUG flwr 2023-04-22 07:02:31,629 | server.py:218 | fit_round 1: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=27363) [Client 2] fit, config: {}
(launch_and_fit pid=27364) [Client 8] fit, config: {}
(launch_and_fit pid=27355) [Client 3] fit, config: {}


DEBUG flwr 2023-04-22 07:02:35,233 | server.py:232 | fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-04-22 07:02:35,238 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-04-22 07:02:35,240 | server.py:168 | evaluate_round 1: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=27364) Epoch 1: train loss 0.06468174606561661, accuracy 0.23
(launch_and_fit pid=27355) Epoch 1: train loss 0.06472698599100113, accuracy 0.21888888888888888
(launch_and_fit pid=27363) Epoch 1: train loss 0.06544194370508194, accuracy 0.22022222222222224
(launch_and_evaluate pid=27363) [Client 3] evaluate, config: {}
(launch_and_evaluate pid=27364) [Client 5] evaluate, config: {}
(launch_and_evaluate pid=27355) [Client 6] evaluate, config: {}


...

DEBUG flwr 2023-04-22 07:02:41,114 | server.py:182 | evaluate_round 2 received 3 results and 0 failures
DEBUG flwr 2023-04-22 07:02:41,115 | server.py:218 | fit_round 3: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=27364) Epoch 1: train loss 0.7796672582626343, accuracy 0.2682222222222222
(launch_and_fit pid=27355) Epoch 1: train loss 0.760401725769043, accuracy 0.27266666666666667
(launch_and_evaluate pid=27363) [Client 5] evaluate, config: {}
(launch_and_evaluate pid=27364) [Client 9] evaluate, config: {}
(launch_and_evaluate pid=27355) [Client 4] evaluate, config: {}
(launch_and_fit pid=27363) [Client 2] fit, config: {}
(launch_and_fit pid=27364) [Client 1] fit, config: {}
(launch_and_fit pid=27355) [Client 8] fit, config: {}


DEBUG flwr 2023-04-22 07:02:43,847 | server.py:232 | fit_round 3 received 3 results and 0 failures
DEBUG flwr 2023-04-22 07:02:43,852 | server.py:168 | evaluate_round 3: strategy sampled 3 clients (out of 10)
DEBUG flwr 2023-04-22 07:02:45,407 | server.py:182 | evaluate_round 3 received 3 results and 0 failures
INFO flwr 2023-04-22 07:02:45,407 | server.py:147 | FL finished in 13.77859928199905
INFO flwr 2023-04-22 07:02:45,408 | app.py:218 | app_fit: losses_distributed [(1, 6.251721282958985), (2, 1.7226357905069987), (3, 0.08978998645146687)]
INFO flwr 2023-04-22 07:02:45,408 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-04-22 07:02:45,408 | app.py:220 | app_fit: metrics_distributed {}
INFO flwr 2023-04-22 07:02:45,408 | app.py:221 | app_fit: losses_centralized []
INFO flwr 2023-04-22 07:02:45,408 | app.py:222 | app_fit: metrics_centralized {}


(launch_and_fit pid=27363) Epoch 1: train loss 0.1765015572309494, accuracy 0.15511111111111112
(launch_and_fit pid=27364) Epoch 1: train loss 0.16916421055793762, accuracy 0.15311111111111111
(launch_and_fit pid=27355) Epoch 1: train loss 0.1739998310804367, accuracy 0.15733333333333333
(launch_and_evaluate pid=27363) [Client 9] evaluate, config: {}
(launch_and_evaluate pid=27364) [Client 2] evaluate, config: {}
(launch_and_evaluate pid=27355) [Client 7] evaluate, config: {}


History (loss, distributed):
	round 1: 6.251721282958985
	round 2: 1.7226357905069987
	round 3: 0.08978998645146687

## Server-side parameter **evaluation**

Flower can evaluate the aggregated model on the server-side or on the client-side. Client-side and server-side evaluation are similar in some ways, but different in others.

**Centralized Evaluation** (or *server-side evaluation*) is conceptually simple: it works the same way that evaluation in centralized machine learning does. If there is a server-side dataset that can be used for evaluation purposes, then that's great. We can evaluate the newly aggregated model after each round of training without having to send the model to clients. We're also fortunate in the sense that our entire evaluation dataset is available at all times.

**Federated Evaluation** (or *client-side evaluation*) is more complex, but also more powerful: it doesn't require a centralized dataset and allows us to evaluate models over a larger set of data, which often yields more realistic evaluation results. In fact, many scenarios require us to use **Federated Evaluation** if we want to get representative evaluation results at all. But this power comes at a cost: once we start to evaluate on the client side, we should be aware that our evaluation dataset can change over consecutive rounds of learning if those clients are not always available. Moreover, the dataset held by each client can also change over consecutive rounds. This can lead to evaluation results that are not stable, so even if we would not change the model, we'd see our evaluation results fluctuate over consecutive rounds.

We've seen how federated evaluation works on the client side (i.e., by implementing the `evaluate` method in `FlowerClient`). Now let's see how we can evaluate aggregated model parameters on the server-side:

In [7]:
# The `evaluate` function will be by Flower called after every round
def evaluate(
    server_round: int,
    parameters: fl.common.NDArrays,
    config: Dict[str, fl.common.Scalar],
) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    net = Net().to(DEVICE)
    valloader = valloaders[0]
    set_parameters(net, parameters)  # Update model with the latest parameters
    loss, accuracy = test(net, valloader)
    print(f"Server-side evaluation loss {loss} / accuracy {accuracy}")
    return loss, {"accuracy": accuracy}

In [8]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    evaluate_fn=evaluate,  # Pass the evaluation function
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-04-22 07:02:45,688 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-04-22 07:02:49,495	INFO worker.py:1553 -- Started a local Ray instance.
INFO flwr 2023-04-22 07:02:50,033 | app.py:180 | Flower VCE: Ray initialized with resources: {'GPU': 1.0, 'CPU': 16.0, 'object_store_memory': 39190219161.0, 'accelerator_type:RTX': 1.0, 'memory': 81443844711.0, 'node:172.17.0.2': 1.0}
INFO flwr 2023-04-22 07:02:50,033 | server.py:86 | Initializing global parameters
INFO flwr 2023-04-22 07:02:50,034 | server.py:269 | Using initial parameters provided by strategy
INFO flwr 2023-04-22 07:02:50,034 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-04-22 07:02:50,114 | server.py:91 | initial parameters (loss, other metrics): 0.07376225471496582, {'accuracy': 0.084}
INFO flwr 2023-04-22 07:02:50,114 | server.py:101 | FL starting
DEBUG flwr 2023-04-22 07:02:50,114 | server.py:218 | fit_round 1: strategy sampled 3 clients (out

Server-side evaluation loss 0.07376225471496582 / accuracy 0.084
(launch_and_fit pid=28535) [Client 4] fit, config: {}
(launch_and_fit pid=28527) [Client 0] fit, config: {}
(launch_and_fit pid=28536) [Client 2] fit, config: {}


DEBUG flwr 2023-04-22 07:02:53,759 | server.py:232 | fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-04-22 07:02:53,764 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
INFO flwr 2023-04-22 07:02:53,840 | server.py:119 | fit progress: (1, 0.06406156420707702, {'accuracy': 0.254}, 3.7258102739870083)
DEBUG flwr 2023-04-22 07:02:53,840 | server.py:168 | evaluate_round 1: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=28535) Epoch 1: train loss 0.06578769534826279, accuracy 0.218
(launch_and_fit pid=28527) Epoch 1: train loss 0.06570062786340714, accuracy 0.2308888888888889
(launch_and_fit pid=28536) Epoch 1: train loss 0.06606102734804153, accuracy 0.21755555555555556
Server-side evaluation loss 0.06406156420707702 / accuracy 0.254


DEBUG flwr 2023-04-22 07:02:55,423 | server.py:182 | evaluate_round 1 received 3 results and 0 failures
WARNING flwr 2023-04-22 07:02:55,424 | fedavg.py:274 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2023-04-22 07:02:55,424 | server.py:218 | fit_round 2: strategy sampled 3 clients (out of 10)


...

DEBUG flwr 2023-04-22 07:03:02,574 | server.py:232 | fit_round 3 received 3 results and 0 failures
INFO flwr 2023-04-22 07:03:02,642 | server.py:119 | fit progress: (3, 0.05280293393135071, {'accuracy': 0.39}, 12.528133098996477)
DEBUG flwr 2023-04-22 07:03:02,643 | server.py:168 | evaluate_round 3: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=28535) Epoch 1: train loss 0.05411229655146599, accuracy 0.35733333333333334
(launch_and_fit pid=28527) Epoch 1: train loss 0.05569254606962204, accuracy 0.3402222222222222
(launch_and_fit pid=28536) Epoch 1: train loss 0.054536785930395126, accuracy 0.35733333333333334
Server-side evaluation loss 0.05280293393135071 / accuracy 0.39


DEBUG flwr 2023-04-22 07:03:04,215 | server.py:182 | evaluate_round 3 received 3 results and 0 failures
INFO flwr 2023-04-22 07:03:04,216 | server.py:147 | FL finished in 14.101416721998248
INFO flwr 2023-04-22 07:03:04,216 | app.py:218 | app_fit: losses_distributed [(1, 0.06374757297833761), (2, 0.05778071657816569), (3, 0.052770209630330404)]
INFO flwr 2023-04-22 07:03:04,216 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-04-22 07:03:04,216 | app.py:220 | app_fit: metrics_distributed {}
INFO flwr 2023-04-22 07:03:04,217 | app.py:221 | app_fit: losses_centralized [(0, 0.07376225471496582), (1, 0.06406156420707702), (2, 0.057544283866882324), (3, 0.05280293393135071)]
INFO flwr 2023-04-22 07:03:04,217 | app.py:222 | app_fit: metrics_centralized {'accuracy': [(0, 0.084), (1, 0.254), (2, 0.304), (3, 0.39)]}


(launch_and_evaluate pid=28535) [Client 5] evaluate, config: {}
(launch_and_evaluate pid=28527) [Client 4] evaluate, config: {}
(launch_and_evaluate pid=28536) [Client 9] evaluate, config: {}


History (loss, distributed):
	round 1: 0.06374757297833761
	round 2: 0.05778071657816569
	round 3: 0.052770209630330404
History (loss, centralized):
	round 0: 0.07376225471496582
	round 1: 0.06406156420707702
	round 2: 0.057544283866882324
	round 3: 0.05280293393135071
History (metrics, centralized):
{'accuracy': [(0, 0.084), (1, 0.254), (2, 0.304), (3, 0.39)]}

## Sending/receiving arbitrary values to/from clients

In some situations, we want to configure client-side execution (trainig, evaluation) from the server-side. One example for that is the server asking the clients to train for a certain number of local epochs. Flower provides a way to send configuration values from the server to the clients using a dictionary. Let's look at an example where the clients receive values from the server through the `config` parameter in `fit` (`config` is also available in `evaluate`). The `fit` method receives the configuration dictionary through the `config` parameter and can then read values from this dictionary. In this example, it reads `server_round` and `local_epochs` and uses those values to improve the logging and configure the number of local training epochs:

In [9]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        # Read values from config
        server_round = config["server_round"]
        local_epochs = config["local_epochs"]

        # Use values provided by the config
        print(f"[Client {self.cid}, round {server_round}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=local_epochs)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

So how can we  send this config dictionary from server to clients? The built-in Flower Strategies provide way to do this, and it works similarly to the way server-side evaluation works. We provide a function to the strategy, and the strategy calls this function for every round of federated learning:

In [10]:
def fit_config(server_round: int):
    """Return training configuration dict for each round.

    Perform two rounds of training with one local epoch, increase to two local
    epochs afterwards.
    """
    config = {
        "server_round": server_round,  # The current round of federated learning
        "local_epochs": 1 if server_round < 2 else 2,  #
    }
    return config

Next, we'll just pass this function to the FedAvg strategy before starting the simulation:

In [11]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    evaluate_fn=evaluate,
    on_fit_config_fn=fit_config,  # Pass the fit_config function
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-04-22 07:03:04,854 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-04-22 07:03:08,589	INFO worker.py:1553 -- Started a local Ray instance.
INFO flwr 2023-04-22 07:03:09,090 | app.py:180 | Flower VCE: Ray initialized with resources: {'memory': 81440134554.0, 'node:172.17.0.2': 1.0, 'GPU': 1.0, 'object_store_memory': 39188629094.0, 'CPU': 16.0, 'accelerator_type:RTX': 1.0}
INFO flwr 2023-04-22 07:03:09,091 | server.py:86 | Initializing global parameters
INFO flwr 2023-04-22 07:03:09,091 | server.py:269 | Using initial parameters provided by strategy
INFO flwr 2023-04-22 07:03:09,091 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-04-22 07:03:09,159 | server.py:91 | initial parameters (loss, other metrics): 0.07391011190414429, {'accuracy': 0.072}
INFO flwr 2023-04-22 07:03:09,159 | server.py:101 | FL starting
DEBUG flwr 2023-04-22 07:03:09,159 | server.py:218 | fit_round 1: strategy sampled 3 clients (out

Server-side evaluation loss 0.07391011190414429 / accuracy 0.072
(launch_and_fit pid=29697) [Client 1, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}
(launch_and_fit pid=29695) [Client 5, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}
(launch_and_fit pid=29706) [Client 0, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}


DEBUG flwr 2023-04-22 07:03:12,763 | server.py:232 | fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-04-22 07:03:12,767 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
INFO flwr 2023-04-22 07:03:12,837 | server.py:119 | fit progress: (1, 0.06278548097610473, {'accuracy': 0.266}, 3.677843862998998)
DEBUG flwr 2023-04-22 07:03:12,838 | server.py:168 | evaluate_round 1: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=29697) Epoch 1: train loss 0.06454993784427643, accuracy 0.23644444444444446
(launch_and_fit pid=29695) Epoch 1: train loss 0.0648919865489006, accuracy 0.22711111111111112
(launch_and_fit pid=29706) Epoch 1: train loss 0.06498292088508606, accuracy 0.2351111111111111
Server-side evaluation loss 0.06278548097610473 / accuracy 0.266


DEBUG flwr 2023-04-22 07:03:14,425 | server.py:182 | evaluate_round 1 received 3 results and 0 failures
WARNING flwr 2023-04-22 07:03:14,426 | fedavg.py:274 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2023-04-22 07:03:14,426 | server.py:218 | fit_round 2: strategy sampled 3 clients (out of 10)


...

DEBUG flwr 2023-04-22 07:03:24,130 | server.py:232 | fit_round 3 received 3 results and 0 failures
INFO flwr 2023-04-22 07:03:24,201 | server.py:119 | fit progress: (3, 0.049795527458190915, {'accuracy': 0.422}, 15.041224953980418)
DEBUG flwr 2023-04-22 07:03:24,201 | server.py:168 | evaluate_round 3: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 0.049795527458190915 / accuracy 0.422


DEBUG flwr 2023-04-22 07:03:25,780 | server.py:182 | evaluate_round 3 received 3 results and 0 failures
INFO flwr 2023-04-22 07:03:25,780 | server.py:147 | FL finished in 16.620860589988297
INFO flwr 2023-04-22 07:03:25,780 | app.py:218 | app_fit: losses_distributed [(1, 0.06243948419888814), (2, 0.05310377430915833), (3, 0.04966310254732768)]
INFO flwr 2023-04-22 07:03:25,781 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-04-22 07:03:25,781 | app.py:220 | app_fit: metrics_distributed {}
INFO flwr 2023-04-22 07:03:25,781 | app.py:221 | app_fit: losses_centralized [(0, 0.07391011190414429), (1, 0.06278548097610473), (2, 0.05322354674339294), (3, 0.049795527458190915)]
INFO flwr 2023-04-22 07:03:25,781 | app.py:222 | app_fit: metrics_centralized {'accuracy': [(0, 0.072), (1, 0.266), (2, 0.39), (3, 0.422)]}


(launch_and_fit pid=29697) Epoch 2: train loss 0.049228593707084656, accuracy 0.4242222222222222
(launch_and_fit pid=29695) Epoch 2: train loss 0.048996951431035995, accuracy 0.42933333333333334
(launch_and_fit pid=29706) Epoch 2: train loss 0.04934291914105415, accuracy 0.4262222222222222
(launch_and_evaluate pid=29697) [Client 9] evaluate, config: {}
(launch_and_evaluate pid=29695) [Client 1] evaluate, config: {}
(launch_and_evaluate pid=29706) [Client 3] evaluate, config: {}


History (loss, distributed):
	round 1: 0.06243948419888814
	round 2: 0.05310377430915833
	round 3: 0.04966310254732768
History (loss, centralized):
	round 0: 0.07391011190414429
	round 1: 0.06278548097610473
	round 2: 0.05322354674339294
	round 3: 0.049795527458190915
History (metrics, centralized):
{'accuracy': [(0, 0.072), (1, 0.266), (2, 0.39), (3, 0.422)]}

As we can see, the client logs now include the current round of federated learning (which they read from the `config` dictionary). We can also configure local training to run for one epoch during the first and second round of federated learning, and then for two epochs during the third round.

Clients can also return arbitrary values to the server. To do so, they return a dictionary from `fit` and/or `evaluate`. We have seen and used this concept throughout this notebook without mentioning it explicitly: our `FlowerClient` returns a dictionary containing a custom key/value pair as the third return value in `evaluate`.

## Scaling federated learning

As a last step in this notebook, let's see how we can use Flower to experiment with a large number of clients.

In [12]:
NUM_CLIENTS = 1000

trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

Files already downloaded and verified
Files already downloaded and verified


We now have 1000 partitions, each holding 45 training and 5 validation examples. Given that the number of training examples on each client is quite small, we should probably train the model a bit longer, so we configure the clients to perform 3 local training epochs. We should also adjust the fraction of clients selected for training during each round (we don't want all 1000 clients participating in every round), so we adjust `fraction_fit` to `0.05`, which means that only 5% of available clients (so 50 clients) will be selected for training each round:


In [13]:
def fit_config(server_round: int):
    config = {
        "server_round": server_round,
        "local_epochs": 3,
    }
    return config


strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.025,  # Train on 25 clients (each round)
    fraction_evaluate=0.05,  # Evaluate on 50 clients (each round)
    min_fit_clients=20,
    min_evaluate_clients=40,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    on_fit_config_fn=fit_config,
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-04-22 07:03:26,989 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-04-22 07:03:30,985	INFO worker.py:1553 -- Started a local Ray instance.
INFO flwr 2023-04-22 07:03:31,521 | app.py:180 | Flower VCE: Ray initialized with resources: {'accelerator_type:RTX': 1.0, 'node:172.17.0.2': 1.0, 'CPU': 16.0, 'object_store_memory': 39195242496.0, 'GPU': 1.0, 'memory': 81455565824.0}
INFO flwr 2023-04-22 07:03:31,524 | server.py:86 | Initializing global parameters
INFO flwr 2023-04-22 07:03:31,524 | server.py:269 | Using initial parameters provided by strategy
INFO flwr 2023-04-22 07:03:31,524 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-04-22 07:03:31,525 | server.py:101 | FL starting
DEBUG flwr 2023-04-22 07:03:31,525 | server.py:218 | fit_round 1: strategy sampled 25 clients (out of 1000)


(launch_and_fit pid=30889) [Client 411, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=30889) Epoch 1: train loss 0.10243480652570724, accuracy 0.13333333333333333
(launch_and_fit pid=30889) Epoch 2: train loss 0.10198924690485, accuracy 0.15555555555555556
(launch_and_fit pid=30889) Epoch 3: train loss 0.10127224028110504, accuracy 0.15555555555555556
(launch_and_fit pid=30898) [Client 276, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=30898) Epoch 1: train loss 0.10211679339408875, accuracy 0.1111111111111111
(launch_and_fit pid=30898) Epoch 2: train loss 0.1018018051981926, accuracy 0.2
(launch_and_fit pid=30898) Epoch 3: train loss 0.10134734958410263, accuracy 0.15555555555555556
(launch_and_fit pid=30889) [Client 278, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=30889) Epoch 1: train loss 0.10276345163583755, accuracy 0.06666666666666667
(launch_and_fit pid=30889) Epoch 2: tra

DEBUG flwr 2023-04-22 07:03:45,487 | server.py:232 | fit_round 1 received 25 results and 0 failures


(launch_and_fit pid=30892) [Client 876, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=30892) Epoch 1: train loss 0.10244282335042953, accuracy 0.1111111111111111
(launch_and_fit pid=30892) Epoch 2: train loss 0.10193042457103729, accuracy 0.15555555555555556
(launch_and_fit pid=30895) [Client 744, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=30895) Epoch 1: train loss 0.10228879749774933, accuracy 0.08888888888888889
(launch_and_fit pid=30895) Epoch 2: train loss 0.10235629975795746, accuracy 0.13333333333333333
(launch_and_fit pid=30895) Epoch 3: train loss 0.10165901482105255, accuracy 0.13333333333333333
(launch_and_fit pid=30883) Epoch 1: train loss 0.1023847907781601, accuracy 0.15555555555555556
(launch_and_fit pid=30883) Epoch 2: train loss 0.1016554906964302, accuracy 0.2
(launch_and_fit pid=30883) Epoch 3: train loss 0.10109458863735199, accuracy 0.2222222222222222
(launch_and_fit pid=30892) Epoch 3: train 

WARNING flwr 2023-04-22 07:03:45,517 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-04-22 07:03:45,518 | server.py:168 | evaluate_round 1: strategy sampled 50 clients (out of 1000)


...

DEBUG flwr 2023-04-22 07:05:01,652 | server.py:232 | fit_round 3 received 25 results and 0 failures
DEBUG flwr 2023-04-22 07:05:01,682 | server.py:168 | evaluate_round 3: strategy sampled 50 clients (out of 1000)


(launch_and_fit pid=30898) [Client 665, round 3] fit, config: {'server_round': 3, 'local_epochs': 3}
(launch_and_fit pid=30898) Epoch 1: train loss 0.10161934047937393, accuracy 0.17777777777777778
(launch_and_fit pid=30898) Epoch 2: train loss 0.10014481097459793, accuracy 0.26666666666666666
(launch_and_fit pid=30898) Epoch 3: train loss 0.09758497774600983, accuracy 0.26666666666666666
(launch_and_fit pid=30893) [Client 214, round 3] fit, config: {'server_round': 3, 'local_epochs': 3}
(launch_and_fit pid=30893) Epoch 1: train loss 0.10164065659046173, accuracy 0.15555555555555556
(launch_and_fit pid=30893) Epoch 2: train loss 0.10035600513219833, accuracy 0.26666666666666666
(launch_and_fit pid=30893) Epoch 3: train loss 0.09940973669290543, accuracy 0.3111111111111111
(launch_and_fit pid=30890) [Client 912, round 3] fit, config: {'server_round': 3, 'local_epochs': 3}
(launch_and_fit pid=30890) Epoch 1: train loss 0.10221931338310242, accuracy 0.08888888888888889
(launch_and_fit pid

DEBUG flwr 2023-04-22 07:05:26,743 | server.py:182 | evaluate_round 3 received 50 results and 0 failures
INFO flwr 2023-04-22 07:05:26,744 | server.py:147 | FL finished in 115.21880033100024
INFO flwr 2023-04-22 07:05:26,744 | app.py:218 | app_fit: losses_distributed [(1, 0.46032305431365955), (2, 0.45846694374084485), (3, 0.45488732147216787)]
INFO flwr 2023-04-22 07:05:26,744 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-04-22 07:05:26,744 | app.py:220 | app_fit: metrics_distributed {}
INFO flwr 2023-04-22 07:05:26,745 | app.py:221 | app_fit: losses_centralized []
INFO flwr 2023-04-22 07:05:26,745 | app.py:222 | app_fit: metrics_centralized {}


(launch_and_evaluate pid=30891) [Client 642] evaluate, config: {}
(launch_and_evaluate pid=30893) [Client 832] evaluate, config: {}
(launch_and_evaluate pid=30890) [Client 2] evaluate, config: {}
(launch_and_evaluate pid=30888) [Client 240] evaluate, config: {}
(launch_and_evaluate pid=30889) [Client 148] evaluate, config: {}
(launch_and_evaluate pid=30898) [Client 886] evaluate, config: {}
(launch_and_evaluate pid=30892) [Client 382] evaluate, config: {}
(launch_and_evaluate pid=30887) [Client 305] evaluate, config: {}
(launch_and_evaluate pid=30883) [Client 818] evaluate, config: {}
(launch_and_evaluate pid=30896) [Client 812] evaluate, config: {}
(launch_and_evaluate pid=30897) [Client 677] evaluate, config: {}
(launch_and_evaluate pid=30886) [Client 301] evaluate, config: {}
(launch_and_evaluate pid=30894) [Client 962] evaluate, config: {}


History (loss, distributed):
	round 1: 0.46032305431365955
	round 2: 0.45846694374084485
	round 3: 0.45488732147216787

## Recap

In this notebook, we've seen how we can gradually enhance our system by customizing the strategy, initializing parameters on the server side, choosing a different strategy, and evaluating models on the server-side. That's quite a bit of flexibility with so little code, right?

In the later sections, we've seen how we can communicate arbitrary values between server and clients to fully customize client-side execution. With that capability, we built a large-scale Federated Learning simulation using the Flower Virtual Client Engine and ran an experiment involving 1000 clients in the same workload - all in a Jupyter Notebook!

## Next steps

Before you continue, make sure to join the Flower community on Slack: [Join Slack](https://flower.dev/join-slack/)

There's a dedicated `#questions` channel if you need help, but we'd also love to hear who you are in `#introductions`!

The [Flower Federated Learning Tutorial - Part 3 [WIP]](https://flower.dev/docs/tutorial/Flower-3-Building-a-Strategy-PyTorch.html) shows how to build a fully custom `Strategy` from scratch.